<a href="https://colab.research.google.com/github/Matheus3788/PLN_PI/blob/explicacao/PI_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio # instalação dos recursos necessários para criar uma interface visual

In [31]:
# importaççoes das bibliotecas necessárias para manipulação de dados, pré-processamento de texto, interface e vetorização...
import pandas as pd # manipulação de dados
import re #pré-processamento de texto
import unicodedata #pré-processamento de texto
import nltk #pré-processamento de texto
from nltk.corpus import stopwords
nltk.download('stopwords') #garante que as stopwords estejam disponíveis localmente.
from sklearn.feature_extraction.text import CountVectorizer #vetorização de texto
from sklearn.feature_extraction.text import TfidfVectorizer #vetorização de texto
import gradio as gr # interface interativa
from sklearn.model_selection import train_test_split # Divisão dos dados para treinamento e Teste
from sklearn.linear_model import LogisticRegression # Importação da biblioteca para LogisticRegression
from sklearn.metrics import classification_report, accuracy_score # Métricas para ver a performance do modelo


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df_treinamento = pd.read_excel('Base_Pedidos.xlsx', engine='openpyxl') # Carregamento dos dados de pedidos no df de treinamento
df_treinamento['request_normalized'] = df_treinamento['request'] #criação de uma coluna para armazenar versões normalizadas das descrições de pedidos.

Explicações prévias:
O método .apply() do pandas permite aplicar uma função em cada elemento de uma coluna ou DataFrame.

In [33]:
# Criação de um dicionário para mapeamento de caracteres
acentos_para_sem_acento = {
    'á': 'a', 'à': 'a', 'ã': 'a', 'â': 'a', 'ä': 'a',
    'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e',
    'í': 'i', 'ì': 'i', 'î': 'i', 'ï': 'i',
    'ó': 'o', 'ò': 'o', 'õ': 'o', 'ô': 'o', 'ö': 'o',
    'ú': 'u', 'ù': 'u', 'û': 'u', 'ü': 'u',
    'ç': 'c',
    'Á': 'A', 'À': 'A', 'Ã': 'A', 'Â': 'A', 'Ä': 'A',
    'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
    'Í': 'I', 'Ì': 'I', 'Î': 'I', 'Ï': 'I',
    'Ó': 'O', 'Ò': 'O', 'Õ': 'O', 'Ô': 'O', 'Ö': 'O',
    'Ú': 'U', 'Ù': 'U', 'Û': 'U', 'Ü': 'U',
    'Ç': 'C'
}

# Substituição dos caracteres que possuem acento, por caracteres sem acento
def remover_acentos(texto):
    texto_sem_acento = ""
    for char in texto:
        if char in acentos_para_sem_acento:
            texto_sem_acento += acentos_para_sem_acento[char]
        else:
            texto_sem_acento += char
    return texto_sem_acento

# Aplicação na coluna df_treinamento['request_normalized'] que eliminará caracteres epeciais, do texto em minusculo e sem acentos
df_treinamento['request_normalized'] = df_treinamento['request_normalized'].apply(
    lambda x: re.sub(r'[^a-z0-9\s]', '', remover_acentos(x).lower())
)

In [34]:
# depois de previamente tratados, aplicamos o split em cada palavra do texto
df_treinamento['request_normalized'] = df_treinamento['request_normalized'].apply(lambda x: x.split())
df_treinamento['request_normalized']

,request_normalized
0,"[estou, passando, por, uma, situacao, dificil,..."
1,"[minha, casa, foi, destruida, por, uma, enchen..."
2,"[meu, marido, esta, doente, e, sem, condicoes,..."
3,"[gostariamos, de, pedir, ajuda, com, materiais..."
4,"[minha, mae, e, idosa, e, estamos, enfrentando..."
...,...
73,"[minha, casa, foi, destruida, por, uma, enchen..."
74,"[estou, desempregado, e, sem, dinheiro, nao, c..."
75,"[minha, familia, esta, com, dificuldades, mas,..."
76,"[preciso, de, dinheiro, agora, para, realizar,..."


In [35]:
# Criação de um conjunto personalizado de stopwords para retirarmos as palavras desnecessárias
stop_words_manual = {
    'de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'e',
    'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as',
    'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'a', 'seu',
    'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta',
    'eu', 'também', 'so', 'pelo', 'pela', 'ate', 'isso', 'ela', 'entre'
}

# retorna somente as palavras que não estão no conjunto de stopwords
def remove_stopwords(palavras):
  texto_filtrado = []
  for palavra in palavras:
    if palavra not in stop_words_manual:
      texto_filtrado.append(palavra)
  return texto_filtrado

# Aplicação da função remove_stopwords na coluna de requisições normalizadas
df_treinamento['request_normalized'] = df_treinamento['request_normalized'].apply(remove_stopwords)



df_treinamento['request_normalized']

,request_normalized
0,"[estou, passando, situacao, dificil, perdi, me..."
1,"[minha, casa, destruida, enchente, recentement..."
2,"[meu, marido, doente, sem, condicoes, trabalha..."
3,"[gostariamos, pedir, ajuda, materiais, escolar..."
4,"[minha, mae, idosa, estamos, enfrentando, difi..."
...,...
73,"[minha, casa, destruida, enchente, perdemos, t..."
74,"[estou, desempregado, sem, dinheiro, consigo, ..."
75,"[minha, familia, dificuldades, verdade, estou,..."
76,"[preciso, dinheiro, agora, realizar, sonho, pe..."


In [26]:
#Depois de tratarmos o texto, retirando caracteres especiais, deixando tudo minusculo e retirando as stopwords, ralizamos a união do texto. O lambda transforma essa lista em uma string única, com as palavras unidas por um espaço (' ') entre elas
df_treinamento['request_normalized_str'] = df_treinamento['request_normalized'].apply(lambda x: ' '.join(x)) # x é cada elemento da coluna request_normalized

vectorizer = CountVectorizer(max_features=1000) #O parâmetro max_features=1000 limita o número máximo de palavras (ou "features") que serão consideradas no vocabulário. Apenas as 1.000 palavras mais frequentes nos dados serão incluídas no modelo. Isso é útil porque textos reais muitas vezes possuem milhares ou até milhões de palavras.
vector = vectorizer.fit_transform(df_treinamento['request_normalized_str']) # temos a criação de uma matriz esparsa
X = pd.DataFrame.sparse.from_spmatrix(vector, columns=vectorizer.get_feature_names_out()) # conversão de uma matriz esparsa em uma matriz de zeros e uns que indicam a recorrência de aparição de uma palavra em um texto

y = df_treinamento['truthfulness'] # a catalogação prévia dos textos com 0 e 1 é utilizada como o eixo 1 para o treinamento


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Divisão do texto em conjunto de texto e de teste, o que pe o random_state

# Métricas dos conjuntos
print("Dimensões do conjunto de treino:", X_train.shape)
print("Dimensões do conjunto de teste:", X_test.shape)
print("Rótulos de treino:", y_train.shape)
print("Rótulos de teste:", y_test.shape)


Dimensões do conjunto de treino: (62, 552)
Dimensões do conjunto de teste: (16, 552)
Rótulos de treino: (62,)
Rótulos de teste: (16,)


In [27]:
#Treinamento do modelo de regressão logística com os dados de treino, usado para prever categorias ou classes
modelo = LogisticRegression(max_iter=1000, random_state=42) # max_inter = Este parâmetro define o número máximo de iterações que o algoritmo de otimização (como o gradiente descendente) pode realizar antes de parar.
# random_state = Esse parâmetro define o valor da semente aleatória para garantir que os resultados sejam reproduzíveis

#tudo isso foi feito para preparar o modelo para ser treinasdo

modelo.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [28]:
y_pred = modelo.predict(X_test)

#Métricas de para avaliar a performance do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

Acurácia: 1.0
Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         8

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [30]:
def preprocessar_texto(texto):
    texto = remover_acentos(texto.lower())
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    palavras = texto.split()
    palavras = [word for word in palavras if word not in stop_words_manual]
    return ' '.join(palavras)

def classificar_frase(frase):
    frase_preprocessada = preprocessar_texto(frase)
    frase_vectorizada = vectorizer.transform([frase_preprocessada])
    previsao = modelo.predict(frase_vectorizada)

    if previsao[0] == 0:
        return "0 - Pedido considerado falso"
    else:
        return "1 - O pedido parece verídico"


interface = gr.Interface(
    fn=classificar_frase,
    inputs=gr.Textbox(lines=2, placeholder="Digite a descrição do pedido aqui...", label="Descrição do Pedido"),
    outputs=gr.Textbox(label="Classificação"),
    title="Classificador de Pedidos",
    description="Digite uma descrição para verificar se o pedido é verdadeiro ou falso.",
    allow_flagging="never"
)

interface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://adad8e4844bab549aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
